# V20 diffraction experiment - February 2018
**Objective:**  
Plotting and comparing experimental 1D data measured on DENEX detector (.root files) and He3 tubes (.asc files) and results of McStas simulations.

In [ ]:
import os 
from mcstasscript.interface import functions
import numpy as np
import scipp as sc
import plopp as pp
import pprint
import ipywidgets
import dataconfig  # to get paths to data

%matplotlib widget

## McStas simulation

In [ ]:
# access data stored after McStas simulations
assert os.path.isdir(dataconfig.data_mcstas), 'The folder which should contain outputs of McStas simulation does not exist.'

data_to_plot = functions.load_data(dataconfig.data_mcstas)

In [ ]:
# Create a dictionary containing filenames, shape of output data and x, y labels
dict_mcstas_files = {}

for item in data_to_plot:
    file_key = item.metadata.info['filename'].rstrip()
    xlabel = item.metadata.info['xlabel'].rstrip()
    ylabel = item.metadata.info['ylabel'].rstrip()
    type_array = item.metadata.info['type']
    position = [float(term) for term in item.metadata.info['position'].split()]
    
    if 'xylimits' in item.metadata.info.keys():
        axes_limits = [float(term) for term in item.metadata.info['xylimits'].split()]
        
    elif 'xlimits' in item.metadata.info.keys():
        axes_limits = [float(term) for term in item.metadata.info['xlimits'].split()]
        
    start = type_array.find('(') + 1
    end = type_array.find(')', start)
    
    
    if ',' in type_array[start:end]:
        nx_value, ny_value = type_array[start:end].split(',')
        
        dict_mcstas_files[file_key] = ((int(nx_value),
                                        int(ny_value)),
                                       xlabel, 
                                       ylabel,
                                      position,
                                      axes_limits)
        
    else:
        dict_mcstas_files[file_key] = (int(type_array[start:end]), 
                                       xlabel, 
                                       ylabel,
                                       position,
                                      axes_limits)
    
pprint.pprint(dict_mcstas_files)

In [ ]:
# plot 1D data generated by McStas simulation and plot using plopp
selected_filename = 'monitor_Hetube5.dat'

path_to_mcstas1D_file = os.path.join(dataconfig.data_mcstas, selected_filename)

assert os.path.isfile(path_to_mcstas1D_file), \
    'There is an issue with the chosen McStas 1D datafile'

print('metadata of input McStas file:', dict_mcstas_files[selected_filename])

x_mcstas, y_mcstas = np.genfromtxt(path_to_mcstas1D_file, 
                                   usecols=(0,1),unpack=True)

da_mcstas1d = sc.DataArray(
    data=sc.array(
        dims=['tof'],
        values=y_mcstas),
    coords={'tof': sc.array(dims=['tof'], 
                            values=x_mcstas*1000, 
                            unit='ms')}
)

pp.plot(da_mcstas1d, 
        grid=True, 
        title=selected_filename,
        linestyle='solid',
        marker=''
       )

Since the sample used for McStas simulations is Nacalf, the possible ROOT files and He3 tubes to use are:
- Spectrum03
- Spectrum11
- Spectrum12

## He3 data

These are the files with an '.asc' extension.

*Naming of these files:*
-  The `bn4` format is a raw data file (event mode) that can be rebinned. Christian Jacobsen (HZG) wrote us a small python script to generate asci files from it, which also generates the filename. So you are completely right: the number indicates the binning size used to histogram the data. I attach the python script as well.
- `chX` corresponds to the channel # of the 4 tubes.  
    a. There are four tubes for diffraction, *i.e.* four channels. TsDau has a total of eight channels. Two of them were used for beam monitors, two were empty.  
    b.  In our case (Feb 2018), `Ch3` was a beam monitor and `Ch5`, `Ch6`, `Ch7`, `Ch8` were the four diffraction tubes.
- `binX` indicates the number of bins used to histogram the data. 

Only data with 2500 bins were considered for Spectrum3 in order to be consistent for all spectra.

In [ ]:
assert os.path.isdir(dataconfig.data_he3), \
    'The path to the folder which should contain datafiles from He3 tubes does not exist.'

In [ ]:
# Spectrum 3
#he3sp3ch3_x, he3sp3ch3_y = np.genfromtxt(os.path.join(dataconfig.data_he3, 'Spectrum03.bn4ch3_bin2500.asc'), unpack=True)
he3sp3ch5_x, he3sp3ch5_y = np.genfromtxt(os.path.join(dataconfig.data_he3, 'Spectrum03.bn4ch5_bin2500.asc'), unpack=True)
#he3sp3ch6_x, he3sp3ch6_y = np.genfromtxt(os.path.join(dataconfig.data_he3, 'Spectrum03.bn4ch6_bin2500.asc'), unpack=True)
#he3sp3ch7_x, he3sp3ch7_y = np.genfromtxt(os.path.join(dataconfig.data_he3, 'Spectrum03.bn4ch7_bin2500.asc'), unpack=True)
#he3sp3ch8_x, he3sp3ch8_y = np.genfromtxt(os.path.join(dataconfig.data_he3, 'Spectrum03.bn4ch8_bin2500.asc'), unpack=True)

da_h3_sp3 = sc.DataArray(data=sc.array(dims=['tof'], values=he3sp3ch5_y),
                        coords={'tof': sc.array(dims=['tof'], values=he3sp3ch5_x, unit='ms')})

In [ ]:
da_h3_sp3

In [ ]:
# # Spectrum 11
# he3sp11ch3_x, he3sp11ch3_y = np.genfromtxt(os.path.join(dataconfig.data_he3, 'Spectrum11.bn4ch3_bin2500.asc'), unpack=True)
# he3sp11ch5_x, he3sp11ch5_y = np.genfromtxt(os.path.join(dataconfig.data_he3, 'Spectrum11.bn4ch5_bin2500.asc'), unpack=True)
# he3sp11ch6_x, he3sp11ch6_y = np.genfromtxt(os.path.join(dataconfig.data_he3, 'Spectrum11.bn4ch6_bin2500.asc'), unpack=True)
# he3sp11ch7_x, he3sp11ch7_y = np.genfromtxt(os.path.join(dataconfig.data_he3, 'Spectrum11.bn4ch7_bin2500.asc'), unpack=True)
# he3sp11ch8_x, he3sp11ch8_y = np.genfromtxt(os.path.join(dataconfig.data_he3, 'Spectrum11.bn4ch8_bin2500.asc'), unpack=True)

In [ ]:
 # Spectrum 12
# he3sp12ch3_x, he3sp12ch3_y = np.genfromtxt(os.path.join(dataconfig.data_he3, 'Spectrum12.bn4ch3_bin2500.asc'), unpack=True)
# he3sp12ch5_x, he3sp12ch5_y = np.genfromtxt(os.path.join(dataconfig.data_he3, 'Spectrum12.bn4ch5_bin2500.asc'), unpack=True)
# he3sp12ch6_x, he3sp12ch6_y = np.genfromtxt(os.path.join(dataconfig.data_he3, 'Spectrum12.bn4ch6_bin2500.asc'), unpack=True)
# he3sp12ch7_x, he3sp12ch7_y = np.genfromtxt(os.path.join(dataconfig.data_he3, 'Spectrum12.bn4ch7_bin2500.asc'), unpack=True)
# he3sp12ch8_x, he3sp12ch8_y = np.genfromtxt(os.path.join(dataconfig.data_he3, 'Spectrum12.bn4ch8_bin2500.asc'), unpack=True)

## ROOT files
Information about 2D data stored in ROOT files

| Spectrum   | $x_{min}$ | $x_{max}$ | $y_{min}$ | $y_{max}$ |
| ---------- | :-------- | :-------- | :-------- | :-------- |
| Spectrum03 | 0.0       | 1024      | -4608     | 4608      |
| Spectrum11 | 0.0       | 4096      | -4608     | 4608      |
| Spectrum12 | 0.0       | 4096      | -4608     | 4608      |

To express the x-axis in TOF in milliseconds,  
- for spectrum03, $x \rightarrow 6.25 + 71\frac{x}{1024}$
- for spectrum11 and spectrum12, $x \rightarrow 6.25 + 71\frac{x}{4096}$

In [ ]:
# Create dictionary to generate plots
# Each entry corresponds to the spectrum number, the associated root file and the folder to get the data from
dict_root_files = {'Spectrum03': ('Spectrum03_DENEX006_1_18-02-05_0000.root', 'Meas_3'), 
                   'Spectrum11': ('Spectrum11_DENEX006_1_18-02-09_0001.root', 'Meas_1'),
                   'Spectrum12': ('Spectrum12_DENEX006_1_18-02-10_0000.root', 'Meas_1')}

dict_selected_dataset = {
'Spectrum03': ('H_TOF_dsp_after_run_3', 'H_TOF,X1-X2_User_2D2_dsp_after_run_3'),
'Spectrum11': ('H_TOF_User_1D1_dsp_after_run_1', 'H_TOF,X1-X2_User_2D4_dsp_after_run_1'),
'Spectrum12': ('H_TOF_User_1D1_dsp_after_run_1', 'H_TOF,X1-X2_User_2D4_dsp_after_run_1')
}

In [ ]:
import uproot

assert os.path.isdir(dataconfig.data_root), \
    'The path to the folder which should contain ROOT files does not exist.'

In [ ]:
# open one ROOT file and extract only one 1D and one 2D dataset specified in dict_selected_dataset
# The 1D and 2D datasets are stored in a dictionary
# Note the vertical axis of 2D datasets is inverted

dict_output = {}

key_spectrum ='Spectrum03'
    
ROOTfile = dict_root_files[key_spectrum][0]
dir_with_data = dict_root_files[key_spectrum][1]

file_to_open = os.path.join(dataconfig.data_root, ROOTfile)

assert os.path.isfile(file_to_open), 'There is an issue with the chosen ROOT file'

with uproot.open(file_to_open)[dir_with_data] as myFile:
    for key, value in myFile.iterclassnames():
        # 1D line plot
        if 'TH1I' in str(value) and dict_selected_dataset[key_spectrum][0] in str(key): 
         
            key_name = myFile[key].name  
            dict_output[key_spectrum+'_1D'] = myFile[key].counts(False)
            
        # 2D contourplot
        elif 'TH2' in str(value) and dict_selected_dataset[key_spectrum][1] in str(key):
            key_name = myFile[key].name
            dict_output[key_spectrum+'_2D'] = np.flip(myFile[key].counts(False), 1)

In [ ]:
dict_output['Spectrum03_1D']

da_root = sc.DataArray(data=sc.array(dims=['tof'], 
                                     values=dict_output['Spectrum03_1D'].astype('float')),
                      coords={'tof': 6.25 * sc.Unit('ms') + 71 / 1024 * sc.arange('tof', 
                                               0, 
                                               len(dict_output['Spectrum03_1D']), 
                                               unit='ms')})

da_root

# Comparison between data from He3, DENEX and McStas simulation

We use the 1D datasets loaded and modfied in the previous cells.

## Physical units in input datafiles

|        | Time          | Length | Intensity     |
| ------ | :------------ | :----- | :------------ |
| McStas | seconds       | meter  | neutron/s/bin |
| He3    | milliseconds  | ?      | neutron/bin?  |
| ROOT   | ?             | ?      | ?             |

## Physical units in derived 1D Scipp dataarrays

|        | Time          | Intensity     |
| ------ | :------------ | :------------ |
| McStas | milliseconds  | neutron/s/bin |
| He3    | milliseconds  | neutron/bin?  |
| ROOT   | milliseconds  | dimensionless |

But the previous cells have modified the ROOT data with Time in milliseconds

In [ ]:
# Shift ROOT data to have ~0 as background
da_root_shifted = da_root.copy()

da_root_shifted.data -= da_root.data.min()

In [ ]:
# Apply coeff to y values of McStas and ROOT data
coeff_mcstas_y = 1250*1000
coeff_root_y = 0.25

In [ ]:
pp.plot({'root': da_root_shifted * coeff_root_y, 
         'mcstas': da_mcstas1d * coeff_mcstas_y, 
         'he3': da_h3_sp3}, 
        grid=True, 
        marker='', 
        linestyle='solid')